In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from datetime import datetime, timedelta

def fetch_noaa_temperature(api_key, start_date, end_date):
    # NOAA API endpoint for temperature data
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    
    headers = {
        'token': api_key
    }
    
    # Set parameters for the request
    params = {
        'datasetid': 'GHCND',  # Daily summaries dataset
        'datatype': 'TMAX',  # Maximum temperature
        'startdate': start_date,
        'enddate': end_date,
        'limit': 1000,
        'units': 'metric',  # Metric units (Celsius)
        'locationid': 'FIPS:01',  # FIPS code for the United States
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    temperatures = []
    for item in data['results']:
        date = item['date'][:10]  # Keep only the date part
        avg_temp = item['value'] / 10.0  # Convert tenths of degrees Celsius
        temperatures.append((date, avg_temp))
    
    return pd.DataFrame(temperatures, columns=['Date', 'AvgTemp'])

def fetch_forecast_data(api_key, lat=39.8283, lon=-98.5795, days=30):
    url = f"http://api.openweathermap.org/data/2.5/forecast?appid={api_key}&lat={lat}&lon={lon}&units=metric"
    response = requests.get(url)
    data = response.json()
    
    if 'list' in data:
        forecast_dates = []
        forecast_temps = []
        
        for entry in data['list']:
            forecast_time = datetime.fromtimestamp(entry['dt'])
            forecast_dates.append(forecast_time)
            forecast_temps.append(entry['main']['temp'])
        
        return pd.DataFrame({'Date': forecast_dates, 'PredictedAvgTemp': forecast_temps})

# API key and date range
api_key = 'YOUR_API_KEY'
start_date = '2020-06-21'
end_date = '2020-07-21'  # 30 days for historical data

# Fetch historical data
historical_data = fetch_noaa_temperature(api_key, start_date, end_date)

# Convert reference date to datetime object
reference_date = datetime.strptime(start_date, '%Y-%m-%d')

# Fetch forecast data starting from the reference date
forecast_data = fetch_forecast_data(api_key, reference_date)

# Combine DataFrames
combined_data = pd.concat([historical_data, forecast_data]).reset_index(drop=True)

# Animate the data
fig, ax = plt.subplots()

def animate(i):
    ax.clear()
    ax.plot(combined_data['Date'][:i+1], combined_data['AvgTemp'][:i+1], color='blue', label='Historical')
    if i < len(historical_data):
        ax.plot(combined_data['Date'][i:], combined_data['PredictedAvgTemp'][i:], color='orange', linestyle='--', label='Forecast')
    ax.set_xlim(combined_data['Date'].min(), combined_data['Date'].max())
    ax.set_ylim(combined_data['AvgTemp'].min() - 5, combined_data['PredictedAvgTemp'].max() + 5)
    ax.set_title('Average Temperature in Continental U.S. (Summer 2020)')
    ax.set_xlabel('Date')
    ax.set_ylabel('Average Temperature (°C)')
    plt.xticks(rotation=45)
    ax.legend()
    ax.grid()

ani = animation.FuncAnimation(fig, animate, frames=len(combined_data), interval=200)
plt.show()
